In [ ]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img,img_to_array
from keras.models import Sequential
from keras.applications import MobileNetV2, ResNet152, VGG16, EfficientNetB0, InceptionV3
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
import os
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels

def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, target_size=(236, 236))
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(features.shape[0], 236, 236, 3)  # Reshape all images in one go
    return features

TRAIN_DIR = "Data/Train"

train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)


train_features = extract_features(train['image'])

x_train = train_features / 255.0

le = LabelEncoder()
le.fit(train['label'])
y_train = le.transform(train['label'])
y_train = to_categorical(y_train, num_classes=2)

model = Sequential()
# Convolutional layers
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(236, 236, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(1024, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(2048, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.fit(x=x_train, y=y_train, batch_size=25, epochs=20,verbose=2)

AI completed
Real completed


  0%|          | 0/801 [00:00<?, ?it/s]

C:\OJ\Python\OpenCV\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
33/33 - 138s - 4s/step - accuracy: 0.6717 - loss: 1.3867
Epoch 2/20
33/33 - 132s - 4s/step - accuracy: 0.9114 - loss: 0.2410
Epoch 3/20
33/33 - 136s - 4s/step - accuracy: 0.9301 - loss: 0.1966
Epoch 4/20
33/33 - 132s - 4s/step - accuracy: 0.9501 - loss: 0.1530
Epoch 5/20
33/33 - 133s - 4s/step - accuracy: 0.9663 - loss: 0.0923
Epoch 6/20
33/33 - 128s - 4s/step - accuracy: 0.9750 - loss: 0.0547
Epoch 7/20
33/33 - 127s - 4s/step - accuracy: 0.9426 - loss: 0.1398
Epoch 8/20
33/33 - 125s - 4s/step - accuracy: 0.9800 - loss: 0.0538
Epoch 9/20
33/33 - 125s - 4s/step - accuracy: 0.9838 - loss: 0.0407
Epoch 10/20
33/33 - 131s - 4s/step - accuracy: 0.9913 - loss: 0.0295
Epoch 11/20
33/33 - 131s - 4s/step - accuracy: 0.9938 - loss: 0.0288
Epoch 12/20
33/33 - 132s - 4s/step - accuracy: 0.9925 - loss: 0.0379
Epoch 13/20
33/33 - 128s - 4s/step - accuracy: 0.9875 - loss: 0.0400
Epoch 14/20
33/33 - 124s - 4s/step - accuracy: 0.9913 - loss: 0.0223
Epoch 15/20
33/33 - 128s - 4s/step - accura

In [ ]:
# Saving the model to a file
# model.save('cnn_model.h5')  # 'cnn_model.h5' is the file where the model will be saved
# Saving just the weights
# model.save_weights('model_weights.weights.h5')


In [ ]:
# from keras.models import load_model

# # Load the saved model
# model = load_model('cnn_model.h5')


In [ ]:
def test_on_1_img(img_name):
    test_image_path = f"Data/Test/{img_name}.jpg"

    img = load_img(test_image_path, target_size=(236, 236))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    #because model ko 4d data chahiye, so ek  extradimension banaya so 1 img hi hai par is form meh (1,236,235,3)

    img_array = img_array / 255.0
    predictions = model.predict(img_array)

    predicted_class = np.argmax(predictions, axis=1)  #yaha axis=1 ka mtlb hai column wise maximum index kaha aa raha hai
    #yeh np.argmax hame index deta hai
    predicted_label = le.inverse_transform(predicted_class) #index se vapas label
    prediction=predicted_label[0] #yeh ek  list thi['real'] so iska 0 element

    return prediction
# test_on_1_img()

In [ ]:
TEST_DIR='Data/Test'
def test(dir):
    image_name = []
    predicted_labels = []
    img_list=os.listdir(dir)[1:]
    for img_with_ext in img_list:
        img_name=img_with_ext[:-4] # to get the file name and remove the .jpg and .png extension
        try:
            print(img_name,'--->')
            predicted_labels.append(test_on_1_img(img_name))
            image_name.append(img_name)
        except:
            exc_img=img_name
            pass

    return image_name , predicted_labels,exc_img
# exc_img refers to a variable containing the image "image_62"....this image was somehow generating some error still not figured out.So I have used error handling to bypass this exceptional image and generate prediction on leftover images.


In [ ]:
image_name , predicted_labels , exc_img=test(TEST_DIR)
predicted_data = pd.DataFrame()
predicted_data['Id'], predicted_data['Label'] =image_name , predicted_labels

image_1 --->
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
image_10 --->
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
image_100 --->
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
image_101 --->
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
image_102 --->
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
image_103 --->
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
image_104 --->
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
image_105 --->
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
image_106 --->
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
image_107 --->
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
image_108 --->
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
image_109 --->
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
image_11 --->
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
image_110 --->
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
image_111 --->
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
image_112 --->
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
image_113 --->
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
image_114 --->
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
image_115 --->
1/1 ━━━━━━━━━━━━━

In [ ]:
predicted_data

,Id,Label
0,image_1,AI
1,image_10,Real
2,image_100,AI
3,image_101,Real
4,image_102,AI
...,...,...
194,image_95,AI
195,image_96,Real
196,image_97,AI
197,image_98,AI


In [ ]:
# test_on_1_img('image_62')

In [ ]:
# predicted_data.to_csv('final_prediction.csv',index=False) # this function is converting pandas dataframe to csv

In [ ]:
# Extract numbers and sort all the images according to id column

predicted_data['number'] = predicted_data['Id'].str.extract('(\d+)').astype(int) # \d+ this is a regular expression matching one or more digits from the string
predicted_data = predicted_data.sort_values('number').drop('number', axis=1)
predicted_data

,Id,Label
0,image_1,AI
111,image_2,Real
123,image_3,Real
134,image_4,Real
145,image_5,AI
...,...,...
107,image_196,Real
108,image_197,Real
109,image_198,AI
110,image_199,Real


In [ ]:
# predicted_data.to_csv('final_prediction_sorted.csv',index=False)